In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe3+ intake
                                        if 'EX_fe3_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
        print('biomass_strain', biomass_strain)        
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7, 17, 15, 19,  2,  2, 18,  1,  1,  5, 15, 12, 15,  3,  3,  2, 16,
        3, 15, 15, 14, 18, 13, 17, 16, 18, 11,  2, 16, 16]), 1: array([ 4,  7, 16,  5,  1, 13, 17, 12,  9, 15,  3, 16, 18, 10, 18, 20, 13,
        1,  2, 17,  1, 10, 15,  7, 10,  8, 12,  6, 13, 13])}
1
strain_number:  0 12.733333333333333
strain_various:  0 7.7413751290638855
strain_number:  1 10.4
strain_various:  1 5.607138307550474
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7, 17, 15, 19,  2,  2, 18,  1,  1,  5, 15, 12, 15,  3,  3,  2, 16,
        3, 15, 15, 14, 18, 13, 17, 16, 18, 11,  2, 16, 16]), 1: array([ 4,  7, 16,  5,  1, 13, 17, 12,  9, 15,  3, 16, 18, 10, 18, 20, 13,
        1,  2, 17,  1, 10, 15,  7, 10,  8, 12,  6, 13, 13])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862

strain_number:  0 19.866666666666667
strain_various:  0 8.578008056782311
strain_number:  1 7.466666666666667
strain_various:  1 3.095516470998373
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 848.2060494974581, 446.82448363040464, 432.5301877334137, 469.8620338897429]
slip_r:  469.8620338897429
biomass_strain {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.678

strain_number:  0 40.03333333333333
strain_various:  0 18.74296905212429
strain_number:  1 4.233333333333333
strain_various:  1 2.1860669909426123
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 848.2060494974581, 446.82448363040464, 432.5301877334137, 469.8620338897429, 458.11406685477897, 447.7198502830043, 465.7036601592628, 464.9364819182489, 469.45546196019933]
slip_r:  469.45546196019933
biomass_strain {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: 

glc__D_e_1:  -15.024622133200975
glc__D_e_1:  -0.4211822173441462
glc__D_e_1:  -15.701174559365473
glc__D_e_1:  -27.27914734039105
glc__D_e_1:  -1.1266903015352625
glc__D_e_1:  -6.017695036843799
glc__D_e_1:  -25.57912967545137
glc__D_e_1:  -0.9538840330245151
glc__D_e_1:  -10.70352716060219
strain_number:  0 38.5
strain_various:  0 17.01910690958841
strain_number:  1 1.3333333333333333
strain_various:  1 1.4907119849998598
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 848.2060494974581, 446.82448363040464, 432.5301877334137, 469.8620338897429, 458.11406685477897, 447.7198502830043, 465.7036601592628, 464.9364819182489, 469.45546196019933, 424.7254845827934, 373.88883092653697, 410.3264083303264, 362.6167986601515]
slip_r:  362.6167986601515
biomass_strain {0: [0.6784862515713712, 0], 1: [0.6784862515713712, 0], 2: [0.6784862515713712, 0], 3: [0.6784862515713712, 0], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391

glc__D_e_1:  -4.520735965474653
glc__D_e_1:  -5.691045581645885
glc__D_e_1:  -7.494622652494753
glc__D_e_1:  -3.253828622494142
glc__D_e_1:  -17.920991093334425
glc__D_e_1:  -9.281624325120521
glc__D_e_1:  -13.43475811825669
glc__D_e_1:  -37.95779700152383
glc__D_e_1:  -12.727523502719933
glc__D_e_1:  -23.720238559562326
glc__D_e_1:  -35.955453712542784
glc__D_e_1:  -13.045519170973147
glc__D_e_1:  -15.009912439032533
glc__D_e_1:  -8.285434621448749
glc__D_e_1:  -7.055089064626901
glc__D_e_1:  -4.070237703082276
glc__D_e_1:  -8.439453746571358
glc__D_e_1:  -3.2104358036457423
glc__D_e_1:  -2.5695341782062173
glc__D_e_1:  -3.6310809119910203
glc__D_e_1:  -1.9287977601522879
glc__D_e_1:  -0.5571057738924194
glc__D_e_1:  -1.182424622423667
glc__D_e_1:  -1.673643463883496
glc__D_e_1:  -0.5773284592896599
glc__D_e_1:  -3.042222801344353
glc__D_e_1:  -3.365309489045533
glc__D_e_1:  -1.6849636524920002
glc__D_e_1:  -1.8241745069424766
strain_number:  0 11.466666666666667
strain_various:  0 12

glc__D_e_1:  -0.22064908246791792
glc__D_e_1:  -0.1922349501949232
glc__D_e_1:  -1.2299927927985992
glc__D_e_1:  -0.7341941635395253
glc__D_e_1:  -1.380734355578497
glc__D_e_1:  -4.5043480038052275
glc__D_e_1:  -3.5743415472550955
glc__D_e_1:  -1.5385380728289402
glc__D_e_1:  -4.22028004101986
glc__D_e_1:  -1.5793610175579011
glc__D_e_1:  -1.7414460893108528
glc__D_e_1:  -0.7203252893367216
glc__D_e_1:  -0.7625427627831741
strain_number:  0 1.2333333333333334
strain_various:  0 1.7829438827088442
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 848.2060494974581, 446.82448363040464, 432.5301877334137, 469.8620338897429, 458.11406685477897, 447.7198502830043, 465.7036601592628, 464.9364819182489, 469.45546196019933, 424.7254845827934, 373.88883092653697, 410.3264083303264, 362.6167986601515, 324.10689949286984, 256.8481587205538, 276.4397390095406, 241.03615028802102, 238.74797800238977, 195.3128121878122, 182.7840909090909, 217.71212121212122]
sli

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20,  1,  5,  2, 14, 11, 10,  4,  6, 18, 18,  7,  4, 17, 16, 19,  3,
        9,  2, 17, 19, 11, 16, 14,  1, 19,  4, 18, 20, 14]), 1: array([18, 20,  5, 14,  5, 11,  9, 20,  6, 10, 12, 16,  3, 19, 18,  6, 10,
       12, 15,  2, 13,  7, 13,  3,  1,  9,  6,  8, 17, 18])}
1
strain_number:  0 13.1
strain_various:  0 7.888176806672967
strain_number:  1 10.866666666666667
strain_various:  1 5.6552826827862654
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20,  1,  5,  2, 14, 11, 10,  4,  6, 18, 18,  7,  4, 17, 16, 19,  3,
        9,  2, 17, 19, 11, 16, 14,  1, 19,  4, 18, 20, 14]), 1: array([18, 20,  5, 14,  5, 11,  9, 20,  6, 10, 12, 16,  3, 19, 18,  6, 10,
       12, 15,  2, 13,  7, 13,  3,  1,  9,  6,  8, 17, 18])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862

glc__D_e_1:  -20.51058454823292
glc__D_e_1:  -14.40821730092154
glc__D_e_1:  -28.60105048749937
glc__D_e_1:  -0.9213820522146463
glc__D_e_1:  -1.0677065668638461
glc__D_e_1:  -12.442669789982439
glc__D_e_1:  -36.04916728982271
glc__D_e_1:  -0.7334418723933211
glc__D_e_1:  -0.47349205725715227
glc__D_e_1:  -13.957300866091481
glc__D_e_1:  -30.520996449773165
glc__D_e_1:  -0.13598402735208404
glc__D_e_1:  -22.58238657547829
glc__D_e_1:  -39.507079789867994
glc__D_e_1:  -1.1369315795807324
glc__D_e_1:  -0.9132004956033788
glc__D_e_1:  -24.41402583658745
glc__D_e_1:  -16.142036729516132
glc__D_e_1:  -0.01138541723037001
glc__D_e_1:  -24.84683764637638
glc__D_e_1:  -47.33950728493426
glc__D_e_1:  -26.177235331337258
glc__D_e_1:  -31.248749390044853
glc__D_e_1:  -8.21036929729624
glc__D_e_1:  -24.0081934159559
glc__D_e_1:  -0.9101591699579803
glc__D_e_1:  -6.756134867132272
glc__D_e_1:  -17.06448632047182
glc__D_e_1:  -6.434340238297987
strain_number:  0 38.36666666666667
strain_various:  0 

glc__D_e_1:  -1.1975030945305607
glc__D_e_1:  -1.6370483641231943
glc__D_e_1:  -0.7216216928033475
glc__D_e_1:  -0.7252272076546108
glc__D_e_1:  -2.8061059056687685
glc__D_e_1:  -2.714301892601261
glc__D_e_1:  -1.5835673120755835
glc__D_e_1:  -0.3692278497544459
glc__D_e_1:  -2.05959749394834
glc__D_e_1:  -1.9892900555222122
glc__D_e_1:  -2.4440766647766345
glc__D_e_1:  -2.8612172241727936
glc__D_e_1:  -1.42593866510186
glc__D_e_1:  -2.9401906493496925
glc__D_e_1:  -1.5488834650890153
glc__D_e_1:  -0.746902559670245
glc__D_e_1:  -0.11487384380070265
strain_number:  0 4.333333333333333
strain_various:  0 2.947692129257886
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 946.9102120789428, 323.6722187126599, 345.7752521601206, 392.2671048249996, 372.8030778440409, 356.53824820717864, 419.26179833052504, 454.10868578851836, 449.0179142609346, 341.535148182018, 355.5104092448369, 406.8202591498441, 432.51843750840703, 3

In [7]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20,  5,  6,  5,  6, 18,  5,  1, 18, 11, 15,  7, 10, 12, 20, 20,  5,
       12, 14,  2, 10,  1,  4, 15, 13, 17,  7, 15, 14, 17]), 1: array([ 4,  3,  2, 14, 13, 10, 13, 15, 11,  6, 17, 11, 14, 20, 17, 18, 17,
        3, 19,  9, 11,  3,  4, 13,  3, 14,  9,  8, 18, 19])}
1
strain_number:  0 12.733333333333333
strain_various:  0 7.061318731102727
strain_number:  1 11.266666666666667
strain_various:  1 5.615059117132151
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20,  5,  6,  5,  6, 18,  5,  1, 18, 11, 15,  7, 10, 12, 20, 20,  5,
       12, 14,  2, 10,  1,  4, 15, 13, 17,  7, 15, 14, 17]), 1: array([ 4,  3,  2, 14, 13, 10, 13, 15, 11,  6, 17, 11, 14, 20, 17, 18, 17,
        3, 19,  9, 11,  3,  4, 13,  3, 14,  9,  8, 18, 19])}
_2:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 

glc__D_e_1:  -12.081824638482614
glc__D_e_1:  -1.8219701596838247
glc__D_e_1:  -0.5159421918091682
glc__D_e_1:  -14.192492811302895
glc__D_e_1:  -19.178162135405074
glc__D_e_1:  -1.2521347665460851
glc__D_e_1:  -7.894696757115739
glc__D_e_1:  -12.85156627394506
glc__D_e_1:  -0.3898031687017216
glc__D_e_1:  -1.2165524678260198
glc__D_e_1:  -7.427818995151187
glc__D_e_1:  -23.345285861596373
glc__D_e_1:  -15.812141690804523
glc__D_e_1:  -3.811462508159156
glc__D_e_1:  -0.5491068709880573
glc__D_e_1:  -6.605111900723571
glc__D_e_1:  -3.2289133304852484
glc__D_e_1:  -20.6640256116491
glc__D_e_1:  -18.73827002288661
glc__D_e_1:  -17.16112379697906
glc__D_e_1:  -21.00561400268689
glc__D_e_1:  -0.20900395895083967
glc__D_e_1:  -6.050753562366275
glc__D_e_1:  -16.057478542769793
glc__D_e_1:  -0.529246281502374
glc__D_e_1:  -0.6624728245082752
glc__D_e_1:  -3.141097305749161
strain_number:  0 40.733333333333334
strain_various:  0 24.69538868335994
strain_number:  1 1.1
strain_various:  1 0.9780

glc__D_e_1:  -6.5324526389210735
glc__D_e_1:  -3.8717600038377356
glc__D_e_1:  -8.304159118934809
glc__D_e_1:  -2.7451473107552915
glc__D_e_1:  -2.186177630396185
glc__D_e_1:  -3.2058100260057008
glc__D_e_1:  -1.748382886039556
glc__D_e_1:  -1.8299703914331464
glc__D_e_1:  -0.5049793407999438
strain_number:  0 4.666666666666667
strain_various:  0 3.389526351702976
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 659.2213238902478, 315.12932391396015, 363.5465524671406, 442.6939181839998, 480.9541986092683, 525.922003022777, 518.4021976729458, 509.1831231293613, 447.9875508571167, 413.4208206181961, 360.9780675100725, 410.9155495931069, 369.31794679805193, 344.60039565692244, 317.4728906161572, 218.60322505251378, 161.50002447411248, 218.443511198946]
slip_r:  218.443511198946
20
glc__D_e_1:  -2.4373998564876382
glc__D_e_1:  -4.853700571198813
glc__D_e_1:  -3.5932089740848037
glc__D_e_1:  -6.0012840670774015
glc__D_e_1:  -2.7022292209736203
glc__D_

In [8]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9, 11,  3,  9, 13, 20,  7,  8, 13, 12, 16,  3, 11, 12, 12, 14,  2,
        9,  7, 14, 15,  7,  2, 10, 14, 10,  4,  4,  4, 17]), 1: array([18, 12, 11, 18, 14, 15, 13, 12,  2,  5,  5, 15,  6,  5,  2, 12,  9,
        2,  6, 10,  8,  2,  4, 15, 20,  8,  3,  8, 12,  3])}
1
strain_number:  0 11.2
strain_various:  0 5.706137047074843
strain_number:  1 9.166666666666666
strain_various:  1 5.273097339852125
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 9, 11,  3,  9, 13, 20,  7,  8, 13, 12, 16,  3, 11, 12, 12, 14,  2,
        9,  7, 14, 15,  7,  2, 10, 14, 10,  4,  4,  4, 17]), 1: array([18, 12, 11, 18, 14, 15, 13, 12,  2,  5,  5, 15,  6,  5,  2, 12,  9,
        2,  6, 10,  8,  2,  4, 15, 20,  8,  3,  8, 12,  3])}
_2:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.52073377571207

glc__D_e_1:  -17.47007989658353
glc__D_e_1:  -10.585685061039921
glc__D_e_1:  -19.401262854815776
glc__D_e_1:  -20.393195838966946
glc__D_e_1:  -7.814476035380352
strain_number:  0 47.46666666666667
strain_various:  0 16.989670064156304
strain_number:  1 0.5
strain_various:  1 1.0567244989431572
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 587.5954363618837, 339.1981412550298, 342.16220782571406, 533.9916651331125, 567.9488072556958, 470.2212699581121, 438.52122980932404, 374.87399424734264, 404.75270753857944, 451.99870066315043, 402.15038420596477, 332.2604186819679, 289.8795730223545, 304.82273339583423]
slip_r:  304.82273339583423
16
glc__D_e_1:  -11.483844336692197
glc__D_e_1:  -0.6526014490409562
glc__D_e_1:  -10.152488215787212
glc__D_e_1:  -25.395947313521148
glc__D_e_1:  -1.5437136167216217
glc__D_e_1:  -9.566910811067954
glc__D_e_1:  -9.765500861514084
glc__D_e_1:  -14.442644266365715
glc__D_e_1:  -27.645321549885026
glc__D_e_1:  -6.663209053493311
glc__D_e_1:  -31.269017831196

glc__D_e_1:  -5.194968284692926
glc__D_e_1:  -6.166681750465423
glc__D_e_1:  -1.8111997076670852
glc__D_e_1:  -7.241288010712486
glc__D_e_1:  -5.9919314507846675
glc__D_e_1:  -3.5407423274732377
glc__D_e_1:  -9.108193876867697
glc__D_e_1:  -9.302094275067908
glc__D_e_1:  -4.582876366142448
glc__D_e_1:  -13.523598747027354
glc__D_e_1:  -5.91688786187926
strain_number:  0 5.133333333333334
strain_various:  0 4.318435930853773
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 587.5954363618837, 339.1981412550298, 342.16220782571406, 533.9916651331125, 567.9488072556958, 470.2212699581121, 438.52122980932404, 374.87399424734264, 404.75270753857944, 451.99870066315043, 402.15038420596477, 332.2604186819679, 289.8795730223545, 304.82273339583423, 238.9934665343384, 201.11226071043495, 214.62354066478136, 185.3594293435494, 193.80619271079794, 220.15265616736204]
slip_r:  220.15265616736204
22
glc__D_e_1:  -0.004891785943539451
glc__D_e_1:  -0.27302847446

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([16,  2,  4,  6,  3,  7, 17,  7, 18,  6, 10, 18, 18,  1,  6,  4, 14,
        4,  1, 19,  4, 11, 17, 13,  9, 14,  2,  2, 13,  7]), 1: array([ 3, 13,  7, 12,  7, 13,  7,  5,  1, 12, 19,  7,  3,  9,  6,  6, 15,
        9,  4,  7,  8, 20,  4, 11,  7,  6, 14,  7,  9,  1])}
1
strain_number:  0 10.433333333333334
strain_various:  0 7.078998673698294
strain_number:  1 8.4
strain_various:  1 4.644710252893428
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([16,  2,  4,  6,  3,  7, 17,  7, 18,  6, 10, 18, 18,  1,  6,  4, 14,
        4,  1, 19,  4, 11, 17, 13,  9, 14,  2,  2, 13,  7]), 1: array([ 3, 13,  7, 12,  7, 13,  7,  5,  1, 12, 19,  7,  3,  9,  6,  6, 15,
        9,  4,  7,  8, 20,  4, 11,  7,  6, 14,  7,  9,  1])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -15.398625041147259
glc__D_e_1:  -13.453338503242637
glc__D_e_1:  -4.213722466213817
glc__D_e_1:  -12.042541100818855
glc__D_e_1:  -5.151952734251026
glc__D_e_1:  -8.470817479880967
glc__D_e_1:  -12.869682123680507
glc__D_e_1:  -17.100970388796327
glc__D_e_1:  -17.677423641458805
glc__D_e_1:  -12.23092187369476
glc__D_e_1:  -20.551463672093163
glc__D_e_1:  -12.378575830777182
glc__D_e_1:  -6.563176399430326
strain_number:  0 49.4
strain_various:  0 18.03995565404749
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 759.702928650297, 393.58229270729277, 339.8720117601697, 430.5993469250049, 445.63619787823944, 444.6459505406874, 537.6402049704682, 428.1823566863078, 403.48583289636673, 411.81954767692065, 457.7683724916865, 424.2428474711562, 348.1026222193186, 195.9427799945763, 211.62648541156398]
slip_r:  211.62648541156398
17
glc__D_e_1:  -18.830223721837168
glc__D_e_1:  -18.93542973953004
glc__D_e_1:

glc__D_e_1:  -10.656736978148675
glc__D_e_1:  -7.623867223205081
glc__D_e_1:  -8.78661920719522
glc__D_e_1:  -2.3572790009046796
glc__D_e_1:  -3.823845146864954
glc__D_e_1:  -1.0254561268423825
glc__D_e_1:  -0.48541360577281734
glc__D_e_1:  -0.05868161408243844
glc__D_e_1:  -0.07261036024611234
glc__D_e_1:  -0.13925106538931753
glc__D_e_1:  -1.2055333811577533
glc__D_e_1:  -0.6699433796384966
glc__D_e_1:  -1.652716668594646
glc__D_e_1:  -1.9174099651115097
glc__D_e_1:  -1.9473349126892732
glc__D_e_1:  -2.448220949549056
glc__D_e_1:  -3.2383543491170155
glc__D_e_1:  -0.7906327630265342
glc__D_e_1:  -0.018109848463712552
glc__D_e_1:  -0.1074628624560916
glc__D_e_1:  -0.13444923739498416
glc__D_e_1:  -0.04661189690472489
glc__D_e_1:  -1.0841105923994743
glc__D_e_1:  -1.9028709824766965
glc__D_e_1:  -3.605422158729564
glc__D_e_1:  -3.9629162923833645
glc__D_e_1:  -3.0115896259226913
glc__D_e_1:  -2.4917536066638197
strain_number:  0 4.133333333333334
strain_various:  0 3.273462319250912
st

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 15,  9, 17,  8, 10, 11,  1, 20, 10, 10, 15,  3, 15, 19,  5,  2,
        2,  9, 16, 14,  1, 19, 17,  2, 13, 11,  5, 16, 17]), 1: array([ 1,  2, 14, 19, 16,  5,  3, 10, 11, 17, 19,  9, 15, 14,  1,  6, 11,
       14, 17,  8, 12, 15, 17,  4, 20,  6, 17, 19, 20,  7])}
1
strain_number:  0 12.233333333333333
strain_various:  0 7.2557257084013
strain_number:  1 11.633333333333333
strain_various:  1 6.002684584602312
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 15,  9, 17,  8, 10, 11,  1, 20, 10, 10, 15,  3, 15, 19,  5,  2,
        2,  9, 16, 14,  1, 19, 17,  2, 13, 11,  5, 16, 17]), 1: array([ 1,  2, 14, 19, 16,  5,  3, 10, 11, 17, 19,  9, 15, 14,  1,  6, 11,
       14, 17,  8, 12, 15, 17,  4, 20,  6, 17, 19, 20,  7])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5:

glc__D_e_1:  -3.1606479930771276
glc__D_e_1:  -15.828699558671008
glc__D_e_1:  -15.917051928465218
glc__D_e_1:  -1.8948099974423656
glc__D_e_1:  -1.0722732138988602
glc__D_e_1:  -7.007579722966589
glc__D_e_1:  -19.34358822709712
glc__D_e_1:  -3.622887461166032
glc__D_e_1:  -1.1928144654846315
glc__D_e_1:  -10.235541769872121
strain_number:  0 45.13333333333333
strain_various:  0 20.216714756744125
strain_number:  1 1.9
strain_various:  1 1.274100990241093
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 982.4394453585628, 330.1958210133521, 357.39953248505867, 435.26594348633813, 452.7268047114011, 378.11088973750265, 409.56023968014625, 431.88981005460306, 345.4290405655535, 412.64410012537576, 412.7758381385736, 364.88628240255895, 331.37886601847873]
slip_r:  331.37886601847873
15
glc__D_e_1:  -12.61159931886247
glc__D_e_1:  -22.508537494266086
glc__D_e_1:  -0.4086595543991933
glc__D_e_1:  -21.7666557897683
glc__D_e_1:  -18.980599501524186
glc__D_e_1:  -0.8594611582266072
glc__D_e_1:  -9.

glc__D_e_1:  -1.9196228758458362
glc__D_e_1:  -0.13883683397870145
glc__D_e_1:  -1.6820377276809269
glc__D_e_1:  -0.2863719167725604
strain_number:  0 6.4
strain_various:  0 4.848367422820456
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 982.4394453585628, 330.1958210133521, 357.39953248505867, 435.26594348633813, 452.7268047114011, 378.11088973750265, 409.56023968014625, 431.88981005460306, 345.4290405655535, 412.64410012537576, 412.7758381385736, 364.88628240255895, 331.37886601847873, 464.51691534995814, 331.45046820911745, 271.73113855886345, 228.43757880108322, 236.60315961128657]
slip_r:  236.60315961128657
20
glc__D_e_1:  -6.8552337327737565
glc__D_e_1:  -4.5000239000818905
glc__D_e_1:  -6.565563809878618
glc__D_e_1:  -4.218741019528322
glc__D_e_1:  -1.1376395731829132
glc__D_e_1:  -2.7306214038479357
glc__D_e_1:  -0.6039673975552011
glc__D_e_1:  -2.0093437975170048
glc__D_e_1:  -0.33062660474603744
glc__D_e_1:  -0.406574376136033
glc__D

In [8]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 1,  7,  7, 10, 12, 10, 10,  6,  5,  5, 11,  9,  5, 11, 19, 14,  8,
       17, 14,  5, 13, 19, 16,  5, 17,  5, 10, 12,  3, 16]), 1: array([15,  1, 15, 20,  5,  8,  2,  7, 14, 15,  9, 14,  2, 12,  1, 10, 15,
       15, 12, 20,  9,  3, 13, 20, 15,  8,  9,  7, 11,  4])}
1
strain_number:  0 11.766666666666667
strain_various:  0 5.666764705034278
strain_number:  1 10.366666666666667
strain_various:  1 5.5526770320469945
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 1,  7,  7, 10, 12, 10, 10,  6,  5,  5, 11,  9,  5, 11, 19, 14,  8,
       17, 14,  5, 13, 19, 16,  5, 17,  5, 10, 12,  3, 16]), 1: array([15,  1, 15, 20,  5,  8,  2,  7, 14, 15,  9, 14,  2, 12,  1, 10, 15,
       15, 12, 20,  9,  3, 13, 20, 15,  8,  9,  7, 11,  4])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639],

glc__D_e_1:  -20.09664677529426
glc__D_e_1:  -21.458371952256577
glc__D_e_1:  -9.369007007518068
strain_number:  0 40.8
strain_various:  0 17.573464845233758
strain_number:  1 1.2
strain_various:  1 0.7483314773547883
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 717.9977117181063, 304.8944115095431, 370.9925655484866, 411.9412528261214, 478.8327061167469, 425.51844665912006, 378.4004989549495, 419.632237608125, 424.7165219751521, 410.0250717818221, 439.8012956990529, 369.6539357593587, 414.4489153777331]
slip_r:  414.4489153777331
15
glc__D_e_1:  -16.214785133325066
glc__D_e_1:  -11.11656429345364
glc__D_e_1:  -8.310962013448624
glc__D_e_1:  -21.434030200486685
glc__D_e_1:  -11.199733373837471
glc__D_e_1:  -8.459685413551599
glc__D_e_1:  -13.177239527473919
glc__D_e_1:  -0.1847823829267874
glc__D_e_1:  -4.804813132691128
glc__D_e_1:  -14.763546377728161
glc__D_e_1:  -0.25318576729440245
glc__D_e_1:  -0.30957194529327947
glc__D_e_1:  -6.244878454361008
glc__D_e_1:  -19.408384533002476
glc

glc__D_e_1:  -4.866160643362846
strain_number:  0 7.933333333333334
strain_various:  0 10.914618128404166
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 717.9977117181063, 304.8944115095431, 370.9925655484866, 411.9412528261214, 478.8327061167469, 425.51844665912006, 378.4004989549495, 419.632237608125, 424.7165219751521, 410.0250717818221, 439.8012956990529, 369.6539357593587, 414.4489153777331, 381.37775898005145, 346.30306477423346, 182.0133643314782, 178.98198459018852, 189.6503436590623, 182.04838723368138]
slip_r:  182.04838723368138
21
glc__D_e_1:  -29.62008822366528
glc__D_e_1:  -8.165288527897426
glc__D_e_1:  -9.407247223517965
glc__D_e_1:  -3.8357147863259997
glc__D_e_1:  -10.490716267695998
glc__D_e_1:  -4.2362237873975905
glc__D_e_1:  -1.9257071409784916
glc__D_e_1:  -7.0748506402007
glc__D_e_1:  -3.580925788691829
glc__D_e_1:  -5.273631182316592
glc__D_e_1:  -1.9050779493703605
glc__D_e_1:  -1.8537965638460556
glc__D_e_1:  -2.740314

strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 717.9977117181063, 304.8944115095431, 370.9925655484866, 411.9412528261214, 478.8327061167469, 425.51844665912006, 378.4004989549495, 419.632237608125, 424.7165219751521, 410.0250717818221, 439.8012956990529, 369.6539357593587, 414.4489153777331, 381.37775898005145, 346.30306477423346, 182.0133643314782, 178.98198459018852, 189.6503436590623, 182.04838723368138, 204.49987580477597, 277.7984022556391, 129.22268907563026, 102.95454545454545, 98.92857142857143, 48.75, 37.5, 15.0, 15.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is:  {}
The change process of microorganism distribution is:  {1: {0: [0.5090172841113372, 4.072138272890697, 4.072138272890697, 6.108207409336046, 7.1262419775587205, 6.108207409336046, 6.108207409336046, 3.5631209887793602, 3.054103704668023, 3.054103704668023, 6.617224693447383, 5.090172841113372, 3.054103704668023

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 12, 12,  2, 19, 10, 15,  6,  1,  4, 19, 18, 12,  4, 20, 14,  7,
        8, 14,  9, 15,  4,  9, 15,  3,  6,  3, 20, 16, 11]), 1: array([ 7, 16,  5,  2, 12, 12, 16,  9, 18, 19, 18, 17, 12, 18, 11, 15, 18,
        5,  4,  6, 19, 19, 16,  1, 11, 18, 16, 20,  2, 15])}
1
strain_number:  0 12.033333333333333
strain_various:  0 7.054470607746236
strain_number:  1 12.566666666666666
strain_various:  1 5.897362875802106
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 12, 12,  2, 19, 10, 15,  6,  1,  4, 19, 18, 12,  4, 20, 14,  7,
        8, 14,  9, 15,  4,  9, 15,  3,  6,  3, 20, 16, 11]), 1: array([ 7, 16,  5,  2, 12, 12, 16,  9, 18, 19, 18, 17, 12, 18, 11, 15, 18,
        5,  4,  6, 19, 19, 16,  1, 11, 18, 16, 20,  2, 15])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -1.3234771255109194
glc__D_e_1:  -7.420145218502501
glc__D_e_1:  -2.105297224024735
glc__D_e_1:  -7.527645750495875
glc__D_e_1:  -14.015816364750895
glc__D_e_1:  -0.14969734030972504
glc__D_e_1:  -0.04907220452767058
glc__D_e_1:  -9.670784130901593
glc__D_e_1:  -16.744595634475886
glc__D_e_1:  -0.5734969496380913
glc__D_e_1:  -13.837635860926348
strain_number:  0 42.6
strain_various:  0 18.325937902328494
strain_number:  1 3.066666666666667
strain_various:  1 1.896195020443719
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 715.4628035792896, 360.3947265969325, 365.77910729621254, 461.99934714408386, 517.6103576248313, 488.9168412239857, 505.1112061924993, 521.9601205516577, 459.7810546425679, 440.65083475886274, 407.17041184418275, 379.88160184907736, 355.75818783567445]
slip_r:  355.75818783567445
15
glc__D_e_1:  -8.212438626692055
glc__D_e_1:  -0.42770645077336233
glc__D_e_1:  -14.25197666299507
glc__D_e_1:  -13.611746693393169
glc__D_e_1:  -0.41430171738450916
glc__D_e_1:  

glc__D_e_1:  -1.9603711076718469
glc__D_e_1:  -1.1709267214543875
glc__D_e_1:  -2.4986907046972004
glc__D_e_1:  -1.1013136513169384
glc__D_e_1:  -1.4559487162945237
glc__D_e_1:  -0.27594409964222955
glc__D_e_1:  -2.129745637381757
glc__D_e_1:  -3.5176286761569937
glc__D_e_1:  -3.297260863519031
glc__D_e_1:  -7.017953959178555
glc__D_e_1:  -2.4363683433871066
glc__D_e_1:  -6.288553574812878
glc__D_e_1:  -2.9356690667326957
glc__D_e_1:  -2.3864321235787203
glc__D_e_1:  -5.777467101762905
glc__D_e_1:  -3.6222932338801797
glc__D_e_1:  -2.083984775250674
glc__D_e_1:  -2.6716347039387878
glc__D_e_1:  -0.8123431516910129
glc__D_e_1:  -0.8780661641538896
strain_number:  0 5.1
strain_various:  0 2.9704096238285613
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 715.4628035792896, 360.3947265969325, 365.77910729621254, 461.99934714408386, 517.6103576248313, 488.9168412239857, 505.1112061924993, 521.9601205516577, 459.7810546425679, 440.65083475886274, 407.

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 14, 13, 12,  6, 13, 13,  2,  1, 14,  3,  6, 10,  3,  1, 17,  7,
       10, 16,  3, 20,  9, 12, 10, 14,  5,  7,  6, 20, 11]), 1: array([12,  2, 16, 11, 16,  3,  8, 19,  6, 19, 18, 18, 12, 13, 20, 14,  1,
       10,  1, 20,  9,  5, 17, 13, 16, 13, 12,  2,  5,  5])}
1
strain_number:  0 11.166666666666666
strain_various:  0 6.31444551977624
strain_number:  1 11.2
strain_various:  1 6.040971224342435
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10, 14, 13, 12,  6, 13, 13,  2,  1, 14,  3,  6, 10,  3,  1, 17,  7,
       10, 16,  3, 20,  9, 12, 10, 14,  5,  7,  6, 20, 11]), 1: array([12,  2, 16, 11, 16,  3,  8, 19,  6, 19, 18, 18, 12, 13, 20, 14,  1,
       10,  1, 20,  9,  5, 17, 13, 16, 13, 12,  2,  5,  5])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -17.22903015168257
glc__D_e_1:  -13.624733460106777
glc__D_e_1:  -18.38285074933101
glc__D_e_1:  -21.034684467502927
glc__D_e_1:  -28.00419185765095
glc__D_e_1:  -0.4549524240342109
glc__D_e_1:  -8.657384354427839
glc__D_e_1:  -6.709489153776428
glc__D_e_1:  -1.7824517868533745
glc__D_e_1:  -16.17768235583997
glc__D_e_1:  -16.007352063484117
glc__D_e_1:  -30.116098855230184
glc__D_e_1:  -19.372100674545813
glc__D_e_1:  -0.702350097832085
glc__D_e_1:  -13.966489009120341
glc__D_e_1:  -20.649649916388423
glc__D_e_1:  -0.03346999499226655
glc__D_e_1:  -14.733839257678413
glc__D_e_1:  -22.111795411897475
glc__D_e_1:  -0.6439428268032099
glc__D_e_1:  -21.19291495203328
glc__D_e_1:  -22.640717835077638
glc__D_e_1:  -0.5041912015508618
glc__D_e_1:  -13.238661176984795
glc__D_e_1:  -39.961425563387124
glc__D_e_1:  -0.24941243272542835
glc__D_e_1:  -17.62262633057133
glc__D_e_1:  -20.47243973560591
glc__D_e_1:  -33.47801800214998
strain_number:  0 38.166666666666664
strain_various:

glc__D_e_1:  -1.4920196219983133
glc__D_e_1:  -3.3201950354386662
glc__D_e_1:  -1.8769240358661738
glc__D_e_1:  -2.2512180243834887
glc__D_e_1:  -2.1685235202542508
glc__D_e_1:  -7.4898922151110146
glc__D_e_1:  -0.5568327028904294
glc__D_e_1:  -2.73342971935723
glc__D_e_1:  -2.7804135563012453
glc__D_e_1:  -3.538956721836094
glc__D_e_1:  -7.170760673987267
glc__D_e_1:  -3.854105628514121
glc__D_e_1:  -3.9146553157269097
glc__D_e_1:  -2.9990872277088343
glc__D_e_1:  -2.873846517678103
glc__D_e_1:  -1.9460721476613267
glc__D_e_1:  -0.34123321054732514
glc__D_e_1:  -1.6944502425301131
glc__D_e_1:  -0.8860696059365355
glc__D_e_1:  -1.5883466376574813
glc__D_e_1:  -2.600880278052181
glc__D_e_1:  -1.244396582709
glc__D_e_1:  -3.438559146131457
glc__D_e_1:  -1.2640057884942406
glc__D_e_1:  -3.3020479641351264
glc__D_e_1:  -0.7539752956703479
glc__D_e_1:  -1.5066307676008421
glc__D_e_1:  -1.4875782172523606
strain_number:  0 4.266666666666667
strain_various:  0 3.182591536608003
strain_number:

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 13, 19,  9, 12,  8,  2, 17,  9, 20,  8,  6,  6,  3,  2, 13,  8,
        2,  6, 20,  7,  8,  8, 10, 20,  7, 11,  3, 13,  7]), 1: array([ 2, 17,  8,  1,  6, 18,  7, 15, 20, 20, 16,  5, 15,  4, 13,  8, 12,
        1, 18, 15,  6,  5,  8, 13, 14,  7, 10,  2, 14, 14])}
1
strain_number:  0 10.733333333333333
strain_various:  0 6.6278872115395036
strain_number:  1 10.466666666666667
strain_various:  1 5.743015081141806
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 13, 19,  9, 12,  8,  2, 17,  9, 20,  8,  6,  6,  3,  2, 13,  8,
        2,  6, 20,  7,  8,  8, 10, 20,  7, 11,  3, 13,  7]), 1: array([ 2, 17,  8,  1,  6, 18,  7, 15, 20, 20, 16,  5, 15,  4, 13,  8, 12,
        1, 18, 15,  6,  5,  8, 13, 14,  7, 10,  2, 14, 14])}
_2:  {0: [0.6703072471157157, 0.2857391744693639], 1: [0.6703072471157157, 0.2857391744693639], 2: [0.6703072471157157, 0.2857391744693639], 3: [0.6703072471157157, 0.2857391744693639], 4: [0.6703072471157157, 0.2857391744693639],

glc__D_e_1:  -24.931535655264952
glc__D_e_1:  -23.740280339688766
glc__D_e_1:  -19.873331315283373
strain_number:  0 43.333333333333336
strain_various:  0 12.111518851444226
strain_number:  1 0.8333333333333334
strain_various:  1 0.8595864638818418
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 790.7650633680047, 306.6128849220955, 317.6870207321059, 410.6487065566014, 417.43101898101895, 424.77660462061993, 467.029368872026, 483.9912709826017, 495.12559986178985, 489.72578091867155, 474.0003270801488, 424.24940376892704, 351.51941711409995, 403.54203382334003]
slip_r:  403.54203382334003
16
glc__D_e_1:  -17.291984856589412
glc__D_e_1:  -19.971240793074855
glc__D_e_1:  -31.288042212028827
glc__D_e_1:  -18.48337132743707
glc__D_e_1:  -16.522839795363815
glc__D_e_1:  -10.838037587599505
glc__D_e_1:  -23.56936649392235
glc__D_e_1:  -9.629303441174601
glc__D_e_1:  -13.017995623224644
glc__D_e_1:  -16.708114565719846
glc__D_e_1:  -9.485227286393263
glc__D_e_1:  -10.37612815808219
glc__D_e_1:  -

glc__D_e_1:  -1.4969008750871913
glc__D_e_1:  -2.39865668595299
glc__D_e_1:  -5.45434838638298
glc__D_e_1:  -4.025193577429715
glc__D_e_1:  -11.906809975458998
glc__D_e_1:  -8.549858002840415
glc__D_e_1:  -8.166514847833902
glc__D_e_1:  -3.277712503204951
glc__D_e_1:  -5.792192431278583
glc__D_e_1:  -4.458905405402696
glc__D_e_1:  -0.8861914242896889
glc__D_e_1:  -1.0370624802177744
glc__D_e_1:  -0.3570570264983345
glc__D_e_1:  -1.3750318306488158
glc__D_e_1:  -0.5110466830519966
glc__D_e_1:  -0.9930993960686992
glc__D_e_1:  -1.2401286269658685
glc__D_e_1:  -1.7971381176352443
glc__D_e_1:  -1.486648447894483
glc__D_e_1:  -0.08330670777469035
glc__D_e_1:  -1.847533897380681
strain_number:  0 4.0
strain_various:  0 3.696845502136472
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115014
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 790.7650633680047, 306.6128849220955, 317.6870207321059, 410.6487065566014, 417.43101898101895, 424.77660462061993, 467.029368872026, 483.9

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  4,  5, 18, 12, 18, 17, 20,  5, 20, 14, 19,  7, 15, 19, 10, 12,
        8,  9,  1, 11, 12, 16, 12, 14, 10, 10, 18,  8, 20]), 1: array([10,  7, 16, 16, 10, 18, 19, 14, 19, 19, 11,  1,  2,  9, 10,  5, 11,
        9, 15, 11,  6, 14, 16,  8,  8, 13, 13, 16, 17,  5])}
1
strain_number:  0 14.3
strain_various:  0 6.558200972827838
strain_number:  1 11.6
strain_various:  1 4.936935621752965
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  4,  5, 18, 12, 18, 17, 20,  5, 20, 14, 19,  7, 15, 19, 10, 12,
        8,  9,  1, 11, 12, 16, 12, 14, 10, 10, 18,  8, 20]), 1: array([10,  7, 16, 16, 10, 18, 19, 14, 19, 19, 11,  1,  2,  9, 10,  5, 11,
        9, 15, 11,  6, 14, 16,  8,  8, 13, 13, 16, 17,  5])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404

glc__D_e_1:  -1.7882230388290372
glc__D_e_1:  -20.255829071423484
glc__D_e_1:  -32.982968477808924
glc__D_e_1:  -2.2100370046653453
glc__D_e_1:  -1.0378728028487796
glc__D_e_1:  -14.302011714137036
glc__D_e_1:  -24.927733975062097
glc__D_e_1:  -1.0878620141391018
glc__D_e_1:  -0.326427365616285
glc__D_e_1:  -6.305712307075874
strain_number:  0 43.766666666666666
strain_various:  0 21.043895921514995
strain_number:  1 2.1666666666666665
strain_various:  1 2.017974782355375
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 569.5815247703794, 370.53811657898336, 409.1525137873822, 384.3442170961134, 420.9184505364471, 397.4965789608114, 382.95220421467934, 455.0415847699563, 432.99395969988285, 359.68597349956366, 323.59386693964296, 405.6824354448553]
slip_r:  405.6824354448553
14
glc__D_e_1:  -8.007757841209688
glc__D_e_1:  -3.5663959074535487
glc__D_e_1:  -16.719694079487176
glc__D_e_1:  -16.994774174478657
glc__D_e_1:  -1.425309822641271
glc__D_e_1:  -1.0256501738358388
glc__D_e_1:  -14.2897

glc__D_e_1:  -7.726511465964573
glc__D_e_1:  -6.9841160418111246
glc__D_e_1:  -7.5433207286315
glc__D_e_1:  -2.4931749265980305
glc__D_e_1:  -3.488957402783587
glc__D_e_1:  -4.183553017232015
glc__D_e_1:  -0.3981040517114301
glc__D_e_1:  -2.9934617352281445
glc__D_e_1:  -1.905047909506698
glc__D_e_1:  -1.9984814849318044
glc__D_e_1:  -1.7750589592112798
strain_number:  0 5.366666666666666
strain_various:  0 4.385455759920766
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 569.5815247703794, 370.53811657898336, 409.1525137873822, 384.3442170961134, 420.9184505364471, 397.4965789608114, 382.95220421467934, 455.0415847699563, 432.99395969988285, 359.68597349956366, 323.59386693964296, 405.6824354448553, 366.5588368074386, 409.91411950708607, 212.5999158286067, 238.9498103842733, 251.58709909634953]
slip_r:  251.58709909634953
19
glc__D_e_1:  -7.024025766013211
glc__D_e_1:  -0.0909662537926259
glc__D_e_1:  -0.6026813313

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  2,  3, 13, 12,  9, 11,  4, 16,  6, 15, 19,  5, 18,  3,  2,  5,
       12, 14,  7,  8, 12,  7, 18,  3, 19, 20, 10, 16,  7]), 1: array([11, 10,  8,  9,  6,  4,  9, 12, 19,  2, 13,  6, 19,  7, 18, 13, 14,
        1, 16, 14,  6, 20,  8,  6, 14,  7,  5,  1,  3, 15])}
1
strain_number:  0 11.833333333333334
strain_various:  0 6.688713545136231
strain_number:  1 9.866666666666667
strain_various:  1 5.432822061834489
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  2,  3, 13, 12,  9, 11,  4, 16,  6, 15, 19,  5, 18,  3,  2,  5,
       12, 14,  7,  8, 12,  7, 18,  3, 19, 20, 10, 16,  7]), 1: array([11, 10,  8,  9,  6,  4,  9, 12, 19,  2, 13,  6, 19,  7, 18, 13, 14,
        1, 16, 14,  6, 20,  8,  6, 14,  7,  5,  1,  3, 15])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.285739174469363

glc__D_e_1:  -3.9775073804802012
glc__D_e_1:  -16.115119245165197
glc__D_e_1:  -22.90902717974956
glc__D_e_1:  -18.368207254739026
glc__D_e_1:  -17.787191072228925
glc__D_e_1:  -20.86440125008695
glc__D_e_1:  -0.24822132869079283
glc__D_e_1:  -11.872878899291194
glc__D_e_1:  -31.198085470585358
glc__D_e_1:  -0.6081526127073786
glc__D_e_1:  -0.7971779644109563
glc__D_e_1:  -11.979930027176735
glc__D_e_1:  -26.486220403192377
glc__D_e_1:  -1.1260017301387641
glc__D_e_1:  -0.7172459378408205
glc__D_e_1:  -23.86797237961084
glc__D_e_1:  -21.94918810583868
glc__D_e_1:  -2.4873576660419463
glc__D_e_1:  -29.562943756650057
glc__D_e_1:  -18.39921222548188
glc__D_e_1:  -0.7231995972716843
glc__D_e_1:  -21.79253919051923
glc__D_e_1:  -31.58547340783262
glc__D_e_1:  -0.9955405499546401
glc__D_e_1:  -9.647963922499095
glc__D_e_1:  -15.253784114753977
glc__D_e_1:  -15.576139146229119
glc__D_e_1:  -21.090199046724372
glc__D_e_1:  -11.563776677613934
glc__D_e_1:  -11.162344055685006
glc__D_e_1:  -24.

glc__D_e_1:  -2.7190160255459204
glc__D_e_1:  -3.4541956049214333
glc__D_e_1:  -4.361026058666299
glc__D_e_1:  -0.8584673910495799
glc__D_e_1:  -1.3311357979883534
glc__D_e_1:  -1.836909561684284
glc__D_e_1:  -0.6189345543210565
glc__D_e_1:  -0.8945337835734888
glc__D_e_1:  -0.18617530847493213
glc__D_e_1:  -0.453528848403295
glc__D_e_1:  -1.8223632393015237
glc__D_e_1:  -0.5039195140969661
strain_number:  0 4.6
strain_various:  0 4.029888335921977
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 764.1970595194281, 336.8842626671573, 355.36269103703324, 403.4776517342305, 444.2110157386474, 420.30495734374824, 351.4189575221197, 400.89628582030963, 462.12727039864615, 411.6005691274779, 359.9499255295873, 336.1409084172709, 380.93769056861987, 368.97195125608954, 287.61311553076507, 215.29462060242994, 255.50363246457792, 206.92008474230016, 198.86048454469505]
slip_r:  198.86048454469505
21
glc__D_e_1:  -7.585936247544868
glc__D_e_1:  -4.76337975

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 19, 10,  1, 12, 13, 19, 13, 20, 15,  5, 15,  4,  3, 14, 16, 12,
        2, 12, 11,  9, 13,  8, 14,  9,  3, 14,  6, 10, 11]), 1: array([ 7, 13,  8,  5,  1, 19, 14,  7,  9,  1, 16,  8,  1,  9,  3, 16, 13,
       14,  3,  2, 20,  5,  1, 10,  3,  9, 17, 14,  9,  7])}
1
strain_number:  0 12.5
strain_various:  0 5.976342248120222
strain_number:  1 8.8
strain_various:  1 5.623759122390171
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10, 19, 10,  1, 12, 13, 19, 13, 20, 15,  5, 15,  4,  3, 14, 16, 12,
        2, 12, 11,  9, 13,  8, 14,  9,  3, 14,  6, 10, 11]), 1: array([ 7, 13,  8,  5,  1, 19, 14,  7,  9,  1, 16,  8,  1,  9,  3, 16, 13,
       14,  3,  2, 20,  5,  1, 10,  3,  9, 17, 14,  9,  7])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.285

glc__D_e_1:  -31.28449535544219
glc__D_e_1:  -9.192786996743216
glc__D_e_1:  -19.373301882848878
glc__D_e_1:  -30.69442466115791
glc__D_e_1:  -11.308157590331001
glc__D_e_1:  -17.23191103526159
glc__D_e_1:  -10.788125719704801
glc__D_e_1:  -5.482897568833513
glc__D_e_1:  -17.392386106750564
glc__D_e_1:  -22.28150951976152
glc__D_e_1:  -14.234134196080367
glc__D_e_1:  -16.899719527324592
glc__D_e_1:  -29.615178832582902
glc__D_e_1:  -11.195277335754291
glc__D_e_1:  -10.414967236365044
glc__D_e_1:  -5.66722582108585
glc__D_e_1:  -0.5591153056028872
glc__D_e_1:  -9.521674537561381
glc__D_e_1:  -11.785381282462513
glc__D_e_1:  -11.449578495905772
glc__D_e_1:  -5.655788811560981
glc__D_e_1:  -2.876825855729667
glc__D_e_1:  -7.774121998506288
glc__D_e_1:  -13.601007057537895
glc__D_e_1:  -1.7261293251126126
glc__D_e_1:  -7.89546939144919
strain_number:  0 43.0
strain_various:  0 21.05389908465096
strain_number:  1 0.13333333333333333
strain_various:  1 0.33993463423951903
fd_r_threshold:  [1

glc__D_e_1:  -0.5768427484676539
glc__D_e_1:  -0.6838499704445362
glc__D_e_1:  -0.27502834088056216
glc__D_e_1:  -0.25799200208330475
glc__D_e_1:  -0.3554571111184407
glc__D_e_1:  -0.0021984571863102875
glc__D_e_1:  -0.7706424761942208
glc__D_e_1:  -1.3567658516037608
glc__D_e_1:  -1.093173096398403
glc__D_e_1:  -2.7600057427938935
glc__D_e_1:  -2.1288013516989657
glc__D_e_1:  -2.084657579364742
glc__D_e_1:  -2.177693020993739
glc__D_e_1:  -1.787284629515792
glc__D_e_1:  -1.8810180917262385
glc__D_e_1:  -2.3107490803871746
glc__D_e_1:  -1.9980156631232227
glc__D_e_1:  -2.1074119162955016
glc__D_e_1:  -1.6883078654425197
glc__D_e_1:  -1.020798108396301
glc__D_e_1:  -0.9399033849015441
glc__D_e_1:  -3.9623016770524337
glc__D_e_1:  -0.6191569314681313
glc__D_e_1:  -1.2006252132974775
glc__D_e_1:  -1.6364904052878608
glc__D_e_1:  -1.1113750506565467
strain_number:  0 2.3666666666666667
strain_various:  0 1.53803626600791
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  9, 12, 20,  7, 13,  8,  6,  6, 16, 17, 18, 13, 14, 12,  9, 20,
       14,  6,  4, 15,  7,  5,  2, 10, 18, 10,  1,  9, 11]), 1: array([ 9,  4, 18,  7, 20, 11, 18,  5,  7, 15,  5, 11, 11, 19,  6, 20, 19,
       14,  6,  8,  6,  9, 11, 12, 18, 13,  2,  7,  9, 20])}
1
strain_number:  0 12.2
strain_various:  0 6.289674077406555
strain_number:  1 11.333333333333334
strain_various:  1 5.44875113112068
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  9, 12, 20,  7, 13,  8,  6,  6, 16, 17, 18, 13, 14, 12,  9, 20,
       14,  6,  4, 15,  7,  5,  2, 10, 18, 10,  1,  9, 11]), 1: array([ 9,  4, 18,  7, 20, 11, 18,  5,  7, 15,  5, 11, 11, 19,  6, 20, 19,
       14,  6,  8,  6,  9, 11, 12, 18, 13,  2,  7,  9, 20])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -10.364642342867135
glc__D_e_1:  -1.0766661785257678
strain_number:  0 45.233333333333334
strain_various:  0 16.51400079393913
strain_number:  1 2.033333333333333
strain_various:  1 1.3780017739062924
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 594.3738158771055, 331.32820359465103, 300.5101126943232, 368.2591177215541, 408.24252874318665, 430.4089800626803, 436.6848973275968, 393.2992204005513, 467.776564826286, 502.2108354734344, 441.1850153484046, 302.7320301042787, 330.3953376462606]
slip_r:  330.3953376462606
15
glc__D_e_1:  -24.80905563874251
glc__D_e_1:  -0.342044815534873
glc__D_e_1:  -35.65455262960751
glc__D_e_1:  -19.941478243650813
glc__D_e_1:  -30.08243974712751
glc__D_e_1:  -19.80852909025774
glc__D_e_1:  -13.616984471763203
glc__D_e_1:  -33.857991288029766
glc__D_e_1:  -0.2835596695023668
glc__D_e_1:  -0.12741404933022205
glc__D_e_1:  -12.795465614924103
glc__D_e_1:  -12.747902751046633
glc__D_e_1:  -20.821097004619155
glc__D_e_1:  -6.974003315579287
glc__D_e

glc__D_e_1:  -2.2457724468804194
glc__D_e_1:  -3.9852120708909586
glc__D_e_1:  -1.7211714923753223
glc__D_e_1:  -1.0599315338327056
glc__D_e_1:  -1.1423441701357484
glc__D_e_1:  -1.8558062460280136
glc__D_e_1:  -1.7690316875656396
glc__D_e_1:  -1.361048108252025
glc__D_e_1:  -1.8069450732549948
glc__D_e_1:  -0.6231562910272719
glc__D_e_1:  -0.28790025890090376
glc__D_e_1:  -0.348145950906268
glc__D_e_1:  -0.5605401952148876
glc__D_e_1:  -0.8896552294821072
glc__D_e_1:  -1.270131026707114
glc__D_e_1:  -0.6033607989461836
glc__D_e_1:  -1.910762009592635
glc__D_e_1:  -2.880253362064149
glc__D_e_1:  -2.6133902618869778
glc__D_e_1:  -1.8737996012836888
glc__D_e_1:  -5.95281897005911
glc__D_e_1:  -8.02200772232102
glc__D_e_1:  -2.4325949303143988
glc__D_e_1:  -6.07834697725943
glc__D_e_1:  -1.9561474196768125
glc__D_e_1:  -3.9442502187618613
glc__D_e_1:  -5.189345144395222
glc__D_e_1:  -2.177989478396859
strain_number:  0 3.5
strain_various:  0 2.5787593916455256
strain_number:  1 0.0
strain

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([12,  6,  4,  9,  7, 18,  1, 12,  8,  7, 15,  1,  1,  5,  5, 11,  1,
       18,  2, 10, 16, 20, 18, 16,  8,  6,  3,  1, 13,  3]), 1: array([ 6,  6, 11, 12, 17, 15, 19, 17,  8, 16,  8, 10, 12,  6,  8,  1, 16,
        9, 16,  4,  9,  6,  6, 20, 10, 13, 16, 17, 13, 18])}
1
strain_number:  0 9.933333333333334
strain_various:  0 7.113055289786208
strain_number:  1 11.5
strain_various:  1 4.937948291885339
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([12,  6,  4,  9,  7, 18,  1, 12,  8,  7, 15,  1,  1,  5,  5, 11,  1,
       18,  2, 10, 16, 20, 18, 16,  8,  6,  3,  1, 13,  3]), 1: array([ 6,  6, 11, 12, 17, 15, 19, 17,  8, 16,  8, 10, 12,  6,  8,  1, 16,
        9, 16,  4,  9,  6,  6, 20, 10, 13, 16, 17, 13, 18])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -18.91896780478436
glc__D_e_1:  -16.454146783786385
glc__D_e_1:  -5.947452125619067
glc__D_e_1:  -12.326998910475389
glc__D_e_1:  -8.265747770223104
glc__D_e_1:  -10.442162903644636
glc__D_e_1:  -13.984523925531843
glc__D_e_1:  -13.27717969055996
glc__D_e_1:  -0.30719934372216384
glc__D_e_1:  -17.11847863640906
glc__D_e_1:  -7.976899025079501
glc__D_e_1:  -0.8755003750572712
strain_number:  0 42.2
strain_various:  0 19.96396754154845
strain_number:  1 0.9
strain_various:  1 1.7953644012660308
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 856.7401552266102, 406.63859787872957, 350.48951538657417, 420.1636543281281, 493.4362859792427, 497.0356883183897, 472.23434418695456, 458.7681505051456, 461.8296197266786, 460.6642784846733, 439.79193080015455, 456.65781464896685, 363.0866169234583, 331.2329309355505, 367.893586906879]
slip_r:  367.893586906879
17
glc__D_e_1:  -5.336543296007449
glc__D_e_1:  -9.667631731482185
glc__D_e_1:  -14.101636717154378
glc__D_e_1:  -7.622884120943794

glc__D_e_1:  -3.814661846008584
glc__D_e_1:  -4.8363427753422545
glc__D_e_1:  -3.5069571225150913
glc__D_e_1:  -2.2663001730878474
glc__D_e_1:  -2.7028927576683754
glc__D_e_1:  -1.7807958949433478
glc__D_e_1:  -1.376648780520539
glc__D_e_1:  -2.5642861701178266
glc__D_e_1:  -3.6603186851256764
glc__D_e_1:  -5.791970566535135
glc__D_e_1:  -7.847481231289376
glc__D_e_1:  -3.280408284929139
glc__D_e_1:  -5.491594485690385
glc__D_e_1:  -5.009900426339912
glc__D_e_1:  -4.471451435444881
glc__D_e_1:  -2.478837004986628
glc__D_e_1:  -1.7148671915346232
glc__D_e_1:  -0.14040473601653902
glc__D_e_1:  -1.1128160402605385
glc__D_e_1:  -1.6056704339571413
glc__D_e_1:  -4.505645519715475
glc__D_e_1:  -2.1855328608914926
strain_number:  0 4.066666666666666
strain_various:  0 3.0433899228035544
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 856.7401552266102, 406.63859787872957, 350.48951538657417, 420.1636543281281, 493.4362859792427, 497.0356883183897, 472.2

In [9]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 10, 13, 19,  2, 19, 15,  8,  9,  9,  4, 10, 17,  9,  4,  7, 20,
       20,  4, 11, 13, 16,  4, 19, 14, 12,  5,  7, 16,  7]), 1: array([ 5,  2,  3, 12, 13,  9,  1, 14, 12,  7,  1, 13, 15, 19,  1,  5,  4,
        9,  9,  2, 14, 14, 13,  1,  2,  3, 15, 17, 18,  6])}
1
strain_number:  0 12.533333333333333
strain_various:  0 6.776101796428057
strain_number:  1 8.633333333333333
strain_various:  1 5.770518944504809
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 10, 13, 19,  2, 19, 15,  8,  9,  9,  4, 10, 17,  9,  4,  7, 20,
       20,  4, 11, 13, 16,  4, 19, 14, 12,  5,  7, 16,  7]), 1: array([ 5,  2,  3, 12, 13,  9,  1, 14, 12,  7,  1, 13, 15, 19,  1,  5,  4,
        9,  9,  2, 14, 14, 13,  1,  2,  3, 15, 17, 18,  6])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5

glc__D_e_1:  -19.737967619583557
glc__D_e_1:  -18.617314324889485
glc__D_e_1:  -24.62237886013618
glc__D_e_1:  -11.378989417223416
glc__D_e_1:  -14.907114557473955
glc__D_e_1:  -18.891238750843026
glc__D_e_1:  -9.921037814074399
glc__D_e_1:  -20.35059994772377
glc__D_e_1:  -18.826271343247328
glc__D_e_1:  -24.58243660008901
glc__D_e_1:  -18.548218017016573
glc__D_e_1:  -9.680778828268927
glc__D_e_1:  -21.997492904738717
glc__D_e_1:  -32.23705920567762
glc__D_e_1:  -13.595639962588113
glc__D_e_1:  -21.206445471920144
glc__D_e_1:  -40.14369987828397
glc__D_e_1:  -18.51938244358032
glc__D_e_1:  -15.00993437802946
glc__D_e_1:  -39.67270493481684
glc__D_e_1:  -23.796370430473704
glc__D_e_1:  -16.41705345332553
glc__D_e_1:  -40.46695868599311
glc__D_e_1:  -0.15223915548669353
glc__D_e_1:  -0.15454026708516722
glc__D_e_1:  -18.001626491545316
strain_number:  0 41.53333333333333
strain_various:  0 16.768488966577227
strain_number:  1 0.1
strain_various:  1 0.30000000000000004
fd_r_threshold:  

glc__D_e_1:  -1.023408608307458
strain_number:  0 4.166666666666667
strain_various:  0 2.130466824169973
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 850.0455846398183, 289.7186672976146, 331.00371558266295, 453.21598795940906, 473.4512103511467, 456.50221659145814, 516.928858775633, 513.8455477345815, 376.6571511810371, 347.6314907628031, 442.8007524285563, 386.3413384890224, 248.31578822119332, 219.3252611731137, 196.36739024754752, 173.0570558371063, 160.8638537341631, 183.9277323553639, 185.4951298701299]
slip_r:  185.4951298701299
21
glc__D_e_1:  -3.142294942151263
glc__D_e_1:  -4.063139962938221
glc__D_e_1:  -1.9110118274901602
glc__D_e_1:  -0.8891549982330686
glc__D_e_1:  -0.0928020786243533
glc__D_e_1:  -0.7551889246701895
glc__D_e_1:  -1.568905841607049
glc__D_e_1:  -1.8089380514331639
glc__D_e_1:  -0.584418099339814
glc__D_e_1:  -1.7207935292988408
glc__D_e_1:  -1.9550634720102997
glc__D_e_1:  -2.1472965987730994
glc__D_e_1:  -1.2979